In [10]:
from youtube_transcript_api import YouTubeTranscriptApi
import yt_dlp

ytt_api = YouTubeTranscriptApi()

transcript = ytt_api.fetch('c4-DIbtNg54', languages = ["es"])

In [15]:

# ' '.join([t.text for t in transcript.snippets])
# transcript.snippets[0]

channel_url='https://www.youtube.com/@atemporalpodcast/videos'

ydl_opts = {
    "quiet": True,
    "extract_flat": True,   # don't download, just get metadata
    "skip_download": True,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(channel_url, download=False)

    # Depending on URL, you may need /videos
    # info["entries"] contains all videos in the channel
    video_ids = [entry["id"] for entry in info["entries"] if entry.get("id")]
    print(video_ids)

len(transcript)

['c4-DIbtNg54', 'go6PLP91MwY', 'lRotUPVcc3s', 'piyxMaLdINI', 'v0UOsdpnTiA', 'xDQ5M7btW64', 'woqzqxRVCHg', 'lCHOK0yNjCc', 'z25ANG83TxA', 'H_hS9UeuaW4', 'nw3cayd9C18', '_ZM5MTi9uLI', '3H_EBWz0D4w', 'M0nD4fgRTsU', 'YhBlPtfGPj4', '9j-QWPMrDhE', 'h40GjfT6ju4', 'HvMIUIF1pHE', 'mPUr6EbZYio', 'v6PpZXlvlH0', '7LD-qvdpHpY', 'SlUryhAHQFI', 'DLKVvlvZZnI', '3eL3wDthCFs', 'KPAscJ6UPek', 'cdWgzqExoa4', 'I1qlJjH9a2s', 'u8QQHYcSvxw', 'yod8trQu59Y', 'r2qpd0TXIa8', 'ngUeXw4wt_M', 'FCQJW5uHvKM', 'qLsy5WCErqw', '_3Ru-34DZyQ', 'mGEkj8porRc', 'yxe7QXEp-ek', '6Uf8NpTmG2c', 'Sh-oa9Ke85o', 'GTSjB4jxTaE', 'qEoftybsMsc', 'RaiWpMxcXv4', 'BQVvnlqwCVs', '6qGbUwDDZWI', 'YnhoN0JEfZo', 'tFke2DHYRS4', 'ohNd-tcrNdo', 'dn6pcctcxWs', '8-XQtwZPowY', 'Pll6UggmPIw', 'qXx5Gykxs4w', 'rQ7HDNqqZ4U', 'rChMRllH72g', 'me_LTZYaWVo', 'KzPzRkd-1rU', 'KNzjq_EncJ4', 'r3CpbRj3vRE', 'Gz_T3oRFZvM', 'ikxZUt168Ts', 'nWHwdasWa2k', 'GQFC18vHlZY', 'hYct7e-PtXg', 'Se-PDh45Wgo', '46EwUiD8EuM', '35KfH5P3bI8', '0LosPaoN-gM', 'yNGUjJnE8g0', '2pMe0re-

4152

In [14]:
len(video_ids)

163